<a href="https://colab.research.google.com/github/gyannetics/nlp-projects-various-usecases/blob/main/FineTune_BERT_model_Document_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
import numpy as np

# Example dataset for Document Classification purpose.
# Each document in the data dictionary contains data that describes a medical conditions (like Condition A, Condition B etc)
data = [
    {"text": "Example text of document related to Condition A", "label": "Condition A"},
    {"text": "Example text of document related to Condition B", "label": "Condition B"},
    # Add more examples as needed
]

# Label mapping
label_map = {"Condition A": 0, "Condition B": 1}

# Tokenizer initialization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_data(data, label_map, tokenizer):
    inputs = tokenizer([doc["text"] for doc in data], padding=True, truncation=True, return_tensors="pt", max_length=512)
    labels = torch.tensor([label_map[doc["label"]] for doc in data])
    return TensorDataset(inputs.input_ids, inputs.attention_mask, labels)

# Encode the dataset
dataset = encode_data(data, label_map, tokenizer)

# Split the dataset into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Model initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))
model.train()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
for epoch in range(3):  # For simplicity, let's train for 3 epochs
    for batch in train_loader:
        batch = tuple(item.to(device) for item in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Save the fine-tuned model and tokenizer
model.save_pretrained("./my_fine_tuned_bert")
tokenizer.save_pretrained("./my_fine_tuned_bert")

# Inference (simplified example)
model.eval()  # Switch to evaluation mode
texts = ["New document text to classify for a medical condition A, as the patients have reported serious concerns"]
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)
with torch.no_grad():
    outputs = model(**encoded_input)
predictions = torch.argmax(outputs.logits, dim=-1)
predicted_labels = [list(label_map.keys())[list(label_map.values()).index(pred)] for pred in predictions.cpu().numpy()]

print("Predicted Labels:", predicted_labels)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.6589480042457581
Epoch 2, Loss: 0.35566282272338867
Epoch 3, Loss: 0.2760579586029053
Predicted Labels: ['Condition A']
